##  Five Day Weather Forecast Information

In [11]:
import requests
import pandas as pd
import numpy as np
import datetime
import json

In [12]:
weather_forecast = requests.get("https://pro.openweathermap.org/data/2.5/forecast/hourly?q=London&appid=9a8ddf0c4ecf94975ce6a1c702afbe2d")
fiveDayJSON = weather_forecast.json()
#fiveDayJSON

## Normalise the JSON file

In [13]:
fiveDayForecast = pd.json_normalize(fiveDayJSON, record_path =['list'])
fiveDayForecast.head()

,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.sea_level,main.grnd_level,main.humidity,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,rain.1h,sys.pod
0,1624975200,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000,0.71,2021-06-29 14:00:00,289.71,289.56,287.79,289.71,1016,1016,1013,82,1.92,90,3.77,9,7.45,0.16,d
1,1624978800,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.64,2021-06-29 15:00:00,289.36,289.23,287.97,289.36,1016,1016,1013,84,1.39,92,3.44,8,7.12,NaN,d
2,1624982400,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.58,2021-06-29 16:00:00,289.41,289.23,288.95,289.41,1016,1016,1013,82,0.46,94,3.23,26,5.85,NaN,d
3,1624986000,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.42,2021-06-29 17:00:00,289.08,288.84,288.66,289.08,1016,1016,1013,81,0.42,96,3.59,26,6.20,NaN,d
4,1624989600,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.42,2021-06-29 18:00:00,287.96,287.72,287.52,287.96,1016,1016,1013,85,0.44,98,3.47,11,6.92,NaN,d


In [15]:
forcast_type = fiveDayForecast[['main.temp','wind.speed', 'main.humidity', 'dt']]
forcast_type.head()

,main.temp,wind.speed,main.humidity,dt
0,289.71,3.77,82,1624975200
1,289.36,3.44,84,1624978800
2,289.41,3.23,82,1624982400
3,289.08,3.59,81,1624986000
4,287.96,3.47,85,1624989600


In [16]:
forcast_fiveday = pd.json_normalize(fiveDayJSON, record_path = ['list', 'weather'])[['main','description']]
forcast_fiveday.head()

,main,description
0,Rain,light rain
1,Clouds,overcast clouds
2,Clouds,overcast clouds
3,Clouds,overcast clouds
4,Clouds,overcast clouds


## Checking whether Rain is Forecast

In [17]:
forecast_list = fiveDayJSON['list']
rain = []

for i in range(len(forecast_list)):
    if 'rain' in forecast_list[i]:
        rain.append(forecast_list[i]['rain'].pop('1h'))
    else:
        rain.append(0)  
df_rain = pd.DataFrame(rain, columns=["Rain"])


In [18]:
fiveDayForecast = pd.concat([forcast_fiveday, df_rain, forcast_type], axis=1)
fiveDayForecast.head()

,Rain,main,description,main.temp,wind.speed,main.humidity,dt
0,0.16,Rain,light rain,289.71,3.77,82,1624975200
1,0.00,Clouds,overcast clouds,289.36,3.44,84,1624978800
2,0.00,Clouds,overcast clouds,289.41,3.23,82,1624982400
3,0.00,Clouds,overcast clouds,289.08,3.59,81,1624986000
4,0.00,Clouds,overcast clouds,287.96,3.47,85,1624989600


In [19]:
fiveDayForecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rain           96 non-null     float64
 1   main           96 non-null     object 
 2   description    96 non-null     object 
 3   main.temp      96 non-null     float64
 4   wind.speed     96 non-null     float64
 5   main.humidity  96 non-null     int64  
 6   dt             96 non-null     int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 5.4+ KB


## Finalised Dataferame

In [20]:
fiveDayForecast['dt'] = pd.to_datetime(fiveDayForecast['dt'], unit='s')
fiveDayForecast = fiveDayForecast.rename(columns={"main.temp": "temp", "wind.speed": "wind_speed", "main.humidity": "humidity"})
fiveDayForecast

,Rain,main,description,temp,wind_speed,humidity,dt
0,0.16,Rain,light rain,289.71,3.77,82,2021-06-29 14:00:00
1,0.00,Clouds,overcast clouds,289.36,3.44,84,2021-06-29 15:00:00
2,0.00,Clouds,overcast clouds,289.41,3.23,82,2021-06-29 16:00:00
3,0.00,Clouds,overcast clouds,289.08,3.59,81,2021-06-29 17:00:00
4,0.00,Clouds,overcast clouds,287.96,3.47,85,2021-06-29 18:00:00
...,...,...,...,...,...,...,...
91,0.87,Rain,light rain,288.64,4.01,96,2021-07-03 09:00:00
92,0.29,Rain,light rain,288.61,4.09,95,2021-07-03 10:00:00
93,0.16,Rain,light rain,288.63,3.89,94,2021-07-03 11:00:00
94,0.22,Rain,light rain,290.32,3.97,89,2021-07-03 12:00:00
